In [1]:
!pip install transformers
!pip install datasets
import gdown
#!pip install google.colab

     |████████████████████████████████| 3.4 MB 5.2 MB/s 
     |████████████████████████████████| 3.3 MB 37.5 MB/s 
     |████████████████████████████████| 596 kB 51.0 MB/s 
     |████████████████████████████████| 61 kB 540 kB/s 
     |████████████████████████████████| 895 kB 49.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 306 kB 5.3 MB/s 
     |████████████████████████████████| 243 kB 45.7 MB/s 
     |████████████████████████████████| 132 kB 50.9 MB/s 
     |████████████████████████████████| 1.1 MB 47.3 MB/s 
     |████████████████████████████████| 160 kB 48.8 MB/s 
     |████████████████████████████████| 271 kB 49.2 MB/s 
     |████████████████████████████████| 192 kB 47.6 MB/s 


In [2]:
import os
!git clone --quiet https://github.com/pistocop/messaging-chat-parser.git
!pip install -q -r messaging-chat-parser/requirements.txt
!git clone --quiet https://github.com/pistocop/pistoBot.git



In [3]:
gdown.download('https://drive.google.com/uc?id=1f4vOzkZfcqzEAo-yujV0KNpGXUsCeQ_W','test.txt',quiet=False)

#https://drive.google.com/file/d/1f4vOzkZfcqzEAo-yujV0KNpGXUsCeQ_W/view?usp=sharing

Downloading...
From: https://drive.google.com/uc?id=1f4vOzkZfcqzEAo-yujV0KNpGXUsCeQ_W
To: /content/test.txt
100%|██████████| 13.8k/13.8k [00:00<00:00, 2.90MB/s]


'test.txt'

In [4]:
!mv test.txt ./messaging-chat-parser/data/chat_raw/whatsapp

In [5]:
whatsapp_user_name = "Maddie_22" # <--- your name, extracted from Whatsapp data
whatsapp_datetime_format = "%m/%d/%y, %H:%M %p" # <-- American format used (MDY)

In [6]:
# Whatsapp
!pwd
!echo $path
print("> [WHATSAPP] start parsing...")
assert whatsapp_user_name is not None, "[!] Whatsapp user name not setted"
!cd messaging-chat-parser/ && python ./src/whatsapp_parser.py --session_token "<|endoftext|>" --user_name $whatsapp_user_name --time_format "$whatsapp_datetime_format"
print("> [WHATSAPP] parsing completed!\n\n")
print("----------------------------------")

# Join Telegram and Whatsapp data
!cd messaging-chat-parser && python ./src/joiner.py
training_data_lines = sum(1 for line in open('./messaging-chat-parser/data/chat_parsed/all-messages.txt'))
print(f"> [PARSER] Training file lines: {training_data_lines}")
print("----------------------------------")

# Check data size
if training_data_lines < 1000:
    print(f"[WARNING] attention insufficient training data ({training_data_lines} < 100K), it is recommended to export more chats")

/content

> [WHATSAPP] start parsing...
[whatsapp_parser.py][INFO]: WA_STOP_WORDS:['https', '<Media omessi>', '<Media omitted>', 'www']
[whatsapp_parser.py][INFO]: Found 1 txt files in `./data/chat_raw/whatsapp/` folder: ['./data/chat_raw/whatsapp/test.txt']
[whatsapp_parser.py][INFO]: Found 28 invalid lines in ./data/chat_raw/whatsapp/test.txt
[whatsapp_parser.py][INFO]: Saving ./data/chat_parsed/wa-chats.txt
> [WHATSAPP] parsing completed!


----------------------------------
[joiner.py][INFO]: files_directory:./data/chat_parsed/ - files_name:['telegram-chats.txt', 'wa-chats.txt']
[joiner.py][WARNING]: File ./data/chat_parsed/telegram-chats.txt provided but not found
[joiner.py][INFO]: N° User messages - 57 messages found. Saving at: ./data/chat_parsed/user-messages.txt
[joiner.py][INFO]: N° Chat messages - 283 messages found. Saving at: ./data/chat_parsed/all-messages.txt
[joiner.py][INFO]: Joiner finished
> [PARSER] Training file lines: 283
----------------------------------
[WARNI

In [7]:
from datasets import load_dataset

raw_datasets = load_dataset("imdb")

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/84.1M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [8]:
import torch
torch.cuda.empty_cache()
!pwd
from torch.utils.data import Dataset
class CustomTextDataset(Dataset):
  def __init__(self, file):
    with open(file,'r') as infile:
      self.text = [line for line in infile]
        
  def __len__(self):
    return len(self.text)

  def __getitem__(self, idx):        
    return {'text':self.text[idx], 'label': 0}


r_j_dataset = CustomTextDataset('./messaging-chat-parser/data/chat_parsed/user-messages.txt')

/content


In [9]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2-medium")
datasets = load_dataset('./messaging-chat-parser/data/chat_parsed')

Downloading:   0%|          | 0.00/718 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Using custom data configuration chat_parsed-66df2c2bcb51c30f


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/chat_parsed-66df2c2bcb51c30f/0.0.0/d86c40dad297bdddf277b406c6a59f0250b5318c400bf23d420a31aff88c84c4. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
#datasets = {'train': r_j_dataset, 'test':r_j_dataset}
tokenizer.pad_token = tokenizer.eos_token
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="longest",  truncation=True)

tokenized_datasets = datasets.map(tokenize_function, batched=True,remove_columns=["text"])
# block_size = tokenizer.model_max_length
block_size = 128

def group_texts(examples):
    #print(examples.keys())
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy() #check predictions 
    return result
    
tokenized_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)


  0%|          | 0/1 [00:00<?, ?ba/s]

In [11]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(100)) 
small_eval_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(100)) 
full_train_dataset = tokenized_datasets["train"]
full_eval_dataset = tokenized_datasets["train"]

Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/text/chat_parsed-66df2c2bcb51c30f/0.0.0/d86c40dad297bdddf277b406c6a59f0250b5318c400bf23d420a31aff88c84c4/cache-155d62f2aab3bb08.arrow


In [12]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("gpt2-medium")

Downloading:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

In [13]:
from transformers import TrainingArguments
torch.cuda.empty_cache()
training_args = TrainingArguments("test_trainer")
#print(str(training_args))
training_args.fp16 = True

from transformers import Trainer

trainer = Trainer(
    model=model, args=training_args, train_dataset=small_train_dataset, eval_dataset=small_eval_dataset
)

trainer.train()

Using amp half precision backend
***** Running training *****
  Num examples = 100
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 39


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=39, training_loss=2.897330846541967, metrics={'train_runtime': 155.2359, 'train_samples_per_second': 1.933, 'train_steps_per_second': 0.251, 'total_flos': 69652552089600.0, 'train_loss': 2.897330846541967, 'epoch': 3.0})

In [14]:
chat = []
start_temperature = 0.1
max_temperature = 3.0

for _ in range(5):
    new_line = "[others] " + input("✍") + '\n'
    chat.append(new_line)
    #ipdb.set_trace(context=6)
    me_token = False
    temperature = start_temperature
    input_network = ' '.join(chat)
    #ipdb.set_trace(context=6)
    while not me_token:
        text = ai.generate(prompt=input_network, 
                           return_as_list=True, 
                           temperature=temperature)
        text = text[0] # batch of 1
        #ipdb.set_trace(context=6)
        text = text.split()
        chat_pos = len(chat)
        #ipdb.set_trace(context=6)
        network_reply = text[chat_pos]
        #ipdb.set_trace(context=6)
        
        if network_reply.startswith('[me]'):
            me_token = True
            network_reply = text[chat_pos] + '\n'
            chat.append(network_reply)
        else:
            if temperature >= max_temperature:
                raise RuntimeError("Max temperature reached")
            temperature += 0.1
            print(f'temperature exit: {temperature}')
    print('Chat:')
    pprint(chat)
    print('---------------------')
    

KeyboardInterrupt: ignored